<a href="https://colab.research.google.com/github/RomaricKanyamibwa/Normalisation-de-concepts-medicaux/blob/master/CNN_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
import logging


In [0]:
!pip3 install --upgrade gensim

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
from gensim.models import Word2Vec
import gensim
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np

In [0]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/Word2VecModels/
!ls -l drive/Coolab/

In [0]:
google_drive="drive/Coolab/"
df_disorders=pd.read_csv("drive/Coolab/Disorders.csv",names=["Code","Disorder","GenDisorder"],sep="\t") 
num_rows=df_disorders.shape[0]

#step 2:Clean data
count_empty=df_disorders.isnull().sum()
count_non_empty=count_empty[count_empty==0]
print(count_non_empty)
#remove empty rows
df_disorders=df_disorders[count_non_empty.keys()]

Code           0
Disorder       0
GenDisorder    0
dtype: int64


In [0]:
GCodes=df_disorders['Code']#Y vector (outputs)
GDisorders=df_disorders['Disorder']#X vector(test)
GenDisorders=df_disorders['GenDisorder']#X vector (train)
print(len(GDisorders))
print(len(GCodes))

623509
623509


In [0]:
Gsample_size=20000#global sample size
sample_size=16000#train size
#train
GDisorders=GDisorders.sample(Gsample_size,random_state=10)
GCodes=GCodes[GDisorders.index]
len(GCodes)


20000

In [0]:
Disorders=GDisorders.sample(sample_size,random_state=10)
Codes=GCodes[Disorders.index]
len(Codes)

16000

In [0]:
#test set
#tsample_size=4000
#testDisorders=df_disorders['Disorder']
testDisorders=GDisorders.drop(Disorders.index)
#testDisorders=testDisorders.sample(tsample_size,random_state=10)
#testCodes=df_disorders['Code']
testCodes=GCodes[testDisorders.index]
len(testCodes)

4000

In [0]:
print(Disorders.index)

Int64Index([244939, 428856, 329881, 494410, 543603, 350144, 323567,  78847,
            495420, 178560,
            ...
            227877, 565009, 186306, 191936,  88999, 569797, 572940, 587735,
            382323, 422008],
           dtype='int64', length=16000)


In [0]:
for i,j in enumerate(zip(Codes,Disorders)):
  print(j)
  if(i>10):break

('C0341679', 'tubular; disease, phosphate-losing')
('C2855139', 'lacerat flexor musc/fasc/tend unsp fngr at wrs/hnd lv, subs')
('C0007760', 'cerebellar diseases [disease/finding]')
('C0161400', 'injury to optic pathways')
('C2892151', 'unsp pedl cyclst injured in collision w hv veh nontraf, subs')
('C2831884', 'penetrating wound w/o foreign body of unsp eyeball, init')
('C1865882', 'arrhythmogenic right ventricular cardiomyopathy 3')
('C0151977', 'intestine small ulcer')
('C0417623', 'accident caused by gases evolved in volcanic eruption')
('C1404510', 'myocarditis; typhus')
('C2834800', 'open bite of unspecified breast, initial encounter')
('C0413711', 'acetaminophen adverse reaction')


In [0]:
for i,j in enumerate(zip(testCodes,testDisorders)):
  print(j)
  if(i>10):break

('C2118738', 'self-inflicted overdose of anterior pituitary hormones')
('C2892793', 'driver of three-wheeled motor vehicle injured in collision with car, pick-up truck or van in nontraffic accident, sequela')
('C0423351', 'lens; calcification')
('C2977177', 'fall from motorized mobility scooter, sequela')
('C3829363', 'kras gln61xxx')
('C1389440', 'abscess; pelvic, female, molar pregnancy')
('C0751847', 'middle cerebral artery embolic infarct')
('C2203991', 'toxicity from propionic acid derivatives due to a self-inflicted overdose')
('C2851623', 'disp fx of neck of fifth mc bone, left hand, init for opn fx')
('C0151772', 'reaction manic')
('C3507105', 'injury of lumbar spinal cord at l5 level with complete lesion of cord')
('C0333400', 'chronic gangrenous inflammation, nos')


In [0]:
def removed_elements_generator(input_list):
    """This function creates an id generator of the empty tokenized elements"""
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess-",i,":",line)
          RemovedElements.append(i)
          #line_process=line.split()
          yield i

def input_tokenize(input_list,RemovedElements):
    """This creates a generator of the tokenized elements """
    
    logging.info("reading file {0}...this may take a while".format(input_list))
    
    for i, line in enumerate (input_list): 
        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        line_process=remove_stopwords(line)
        line_process=gensim.utils.simple_preprocess(line_process,min_len=3,max_len=30)
        if not line_process:
          print(line_process,"before preprocess-",i,":",line)
          RemovedElements.append(i)
          #line_process=line.split()
        else:    
          yield line_process

In [0]:
logging.info ("Tokenization")
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
#GenDisorders_corpus=(input_tokenize(GenDisorders))
logging.info ("End of Tokenization")

In [0]:
for i,s in enumerate(Disorders_corpus):
  #print(s)
  k=1
  #if i >:break
    
print(RemovedElements)

[] before preprocess- 557 : mc5dm2
[] before preprocess- 2634 : ea4
[] before preprocess- 4086 : rp73
[] before preprocess- 4708 : t(2;17)(p23;q35)
[] before preprocess- 6676 : cd332 c.1645a>c
[] before preprocess- 7283 : oi14
[] before preprocess- 8917 : hs5
[] before preprocess- 10456 : dn
[] before preprocess- 12247 : rp15
[] before preprocess- 12580 : ad18
[] before preprocess- 12720 : ad5
[] before preprocess- 13014 : cd332 c.1144t>c
[] before preprocess- 13351 : t(17;21)(q11;q22)
[] before preprocess- 13516 : as
[] before preprocess- 13768 : fire in mine
[] before preprocess- 14060 : ml i
[] before preprocess- 15851 : ts
[] before preprocess- 15897 : oi9
[] before preprocess- 15934 : rp13
[557, 2634, 4086, 4708, 6676, 7283, 8917, 10456, 12247, 12580, 12720, 13014, 13351, 13516, 13768, 14060, 15851, 15897, 15934]


In [0]:
model_dir=google_drive+"Disorders"
model=Word2Vec.load(model_dir)


In [0]:
def getsentence_matrix(sentence,gensim_model,nrows=30):
  #print("sent:",sentence)
  sentence_matrix=[gensim_model.wv.get_vector(token) for token in sentence]
 
  sentence_matrix=np.matrix(sentence_matrix)
#  print("Before padding:")
#  pprint(sentence_matrix.T[:2,:])
  #print("size before padding:",nrows-sentence_matrix.shape[0])
  if(nrows-sentence_matrix.shape[0]<0 or sentence_matrix.shape[1]<=0):
    print("Sentence:",sentence)
    print("MatrixShape:",sentence_matrix.shape)
    print(sentence_matrix)
  sentence_matrix=np.lib.pad(sentence_matrix,((0,max(0,nrows-sentence_matrix.shape[0])),(0,0)), 'constant', constant_values=(0))
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=0)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
  #sentence_matrix=np.delete(sentence_matrix,np.s_[::2],axis=1)
#  print("After padding:")
#  pprint(sentence_matrix.T[:2,:])
  #print("shape=",sentence_matrix.shape)
  return sentence_matrix#.T

In [0]:
def get_termes_matrices(Corpus,model):
  
#  pprint(Corpus)
  matrix_list=[]
  IndexOfEmptyElem=[]
  nrows=30
  empty=True
  for i,sentence in enumerate(Corpus):
    if not(not sentence):
      if empty==True:
        matrix_list=np.array([getsentence_matrix(sentence,model,nrows=30)])
        empty=False
        #print(matrix_list[0].shape)
      else:
        #print(i,":",getsentence_matrix(sentence,model).shape)
        matrix_list=np.append(matrix_list,[getsentence_matrix(sentence,model,nrows=30)],axis=0)
      #print("i:",i)#,matrix_list)
      if(i%1000==0):
        print("i:",i)
       # break
    else:
      print("Empty element's index:",i)
      IndexOfEmptyElem.append(i)
      #print(np.zeros((5,5)))
      matrix_list=np.append(matrix_list,[np.zeros((nrows,model.wv.vector_size))],axis=0)
  print("length:",len(matrix_list))
  print("shape=",matrix_list.shape)
  #print(matrix_list)
  return matrix_list,IndexOfEmptyElem


In [0]:
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
testRemovedElements=[]
testDisorders_corpus=(input_tokenize(testDisorders,testRemovedElements))

In [0]:
RemovedElementsIter=removed_elements_generator(Disorders)
testRemovedElementsIter=removed_elements_generator(testDisorders)

In [0]:
print(list(RemovedElementsIter))
print("Test")
print(list(testRemovedElementsIter))

In [0]:
RemovedElements=[]
Disorders_corpus=(input_tokenize(Disorders,RemovedElements))
testRemovedElements=[]
testDisorders_corpus=(input_tokenize(testDisorders,testRemovedElements))


In [0]:
MatrixList,empty=get_termes_matrices(Disorders_corpus,model)

i: 0
[] before preprocess- 557 : mc5dm2
i: 1000
i: 2000
[] before preprocess- 2634 : ea4
i: 3000
i: 4000
[] before preprocess- 4086 : rp73
[] before preprocess- 4708 : t(2;17)(p23;q35)
i: 5000
i: 6000
[] before preprocess- 6676 : cd332 c.1645a>c
i: 7000
[] before preprocess- 7283 : oi14
i: 8000
[] before preprocess- 8917 : hs5
i: 9000
i: 10000
[] before preprocess- 10456 : dn
i: 11000
i: 12000
[] before preprocess- 12247 : rp15
[] before preprocess- 12580 : ad18
[] before preprocess- 12720 : ad5
i: 13000
[] before preprocess- 13014 : cd332 c.1144t>c
[] before preprocess- 13351 : t(17;21)(q11;q22)
[] before preprocess- 13516 : as
[] before preprocess- 13768 : fire in mine
i: 14000
[] before preprocess- 14060 : ml i
i: 15000
[] before preprocess- 15851 : ts
[] before preprocess- 15897 : oi9
[] before preprocess- 15934 : rp13
length: 15981
shape= (15981, 30, 300)


In [0]:
print("test sample")
testMatrixList,testempty=get_termes_matrices(testDisorders_corpus,model)

test sample
i: 0
[] before preprocess- 230 : t(17;22)(q22;q13)
[] before preprocess- 232 : rp69
i: 1000
[] before preprocess- 1730 : t2d
[] before preprocess- 1836 : k20r
i: 2000
[] before preprocess- 2974 : md1
i: 3000
[] before preprocess- 3354 : f2 20210g-a
[] before preprocess- 3567 : t(7;10)(q34;q24)
[] before preprocess- 3949 : a2md
[] before preprocess- 3993 : pc2
length: 3991
shape= (3991, 30, 300)


In [0]:
def remove_element_byIndex(panda_dataset,listOfIndexes):
  Indexes=[]
  for elem in listOfIndexes:
    Indexes.append(panda_dataset[elem:elem+1].index[0])
  
  print(Indexes)
  return panda_dataset.drop(Indexes)

In [0]:


print(Codes.reindex([219126, 267602]))
#print(Disorders[RemovedElements[0]:RemovedElements[0]+1])
print(Codes[RemovedElements[0]:RemovedElements[0]+1])
#print(Disorders[RemovedElements[1]:RemovedElements[1]+1])
#po=pd.Int64Index([], dtype='int64')
#po.append(224267)

print("Before drop:",len(Codes))
Codes2=remove_element_byIndex(Codes,RemovedElements)
print("After drop:",len(Codes2))
print(testempty)
print("after drop")

219126    C3810041
267602    C2675192
Name: Code, dtype: object
224267    C3275685
Name: Code, dtype: object
Before drop: 16000
[224267, 110224, 151841, 30167, 24110, 304425, 267602, 315505, 231113, 219126, 314043, 26207, 23305, 285892, 381658, 124679, 323019, 125081, 244552]
After drop: 15981
[]
after drop


In [0]:
print("Before drop:",len(testCodes))
testCodes2=remove_element_byIndex(testCodes,testRemovedElements)
print("After drop:",len(testCodes2))

Before drop: 4000
[26735, 284050, 67531, 29851, 563840, 23240, 31064, 216601, 298433]
After drop: 3991


In [0]:
print("Before drop:",len(GCodes))
GCodes2=GCodes.drop(Codes2.index)
print("After drop:",len(GCodes2))
GCodes2=GCodes2.drop(testCodes2.index)
print("After drop:",len(GCodes2))
GCodes2=GCodes.drop(GCodes2.index)
print("After Final Global drop:",len(GCodes2))


Before drop: 20000
After drop: 4019
After drop: 28
After Final Global drop: 19972


In [0]:
def get_code_classes(Codes,gensim_model):
  unique_Codes=list(set(Codes))
  nb_classes=len(unique_Codes)
  print("number of codes:",nb_classes)
  Code_vectors=[gensim_model.wv.get_vector(token) for token in Codes]
  Code_dict=dict(zip(Codes,Code_vectors))
  #inv_Code_dict=dict(zip(Code_vectors,Codes))
  #print(Code_vectors[0][0:5])
  #print("'C0009021'",Code_dict['C0009021'][0:5])
  return nb_classes,Code_vectors,Code_dict

In [0]:
nb_classes,Code_vectors,Code_dict=get_code_classes(Codes2,model)

number of codes: 14912


In [0]:
print(Codes[222160])
print("Dict:",Code_dict["C0275585"][0:5])
print("Vect:",Code_vectors[0][0:5])

C0275585
Dict: [-0.08187973 -0.05139588 -0.13897417 -0.08761195 -0.03717935]
Vect: [-0.028957    0.01374186  0.1307545   0.02424315  0.04272182]


In [0]:
#testnb_classes,testCode_vectors,testCode_dict=get_code_classes(testCodes2,model)

Num_classes,GCode_vectors,GCode_dict=get_code_classes(GCodes2,model)

number of codes: 18372


In [0]:
frames = [Codes2,testCodes2]
result = pd.concat(frames)

print(len(((result))))

print( len( result[Codes2.index] ) )
print(result[0:len(Codes2)+1])

In [0]:
print(result)

In [0]:
batch_size = 128
num_classes = nb_classes
epochs = 10

# input dimensions
size_x, size_y = MatrixList[0].shape
keras_model = Sequential()
print(type(np.array(MatrixList)))

# load the MNIST data set, which already splits into train and test sets for us
x_train, y_train = np.array(MatrixList),np.array(Codes2)
x_test, y_test = np.array(testMatrixList),np.array(testCodes2)
print(type(x_train[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [0]:
print((x_train.shape))
print((x_test.shape))
print((y_train.shape))
print((y_test.shape))

(15981, 30, 300)
(3991, 30, 300)
(15981,)
(3991,)


In [0]:
# reshape the data into a 4D tensor - (sample_number, x_size, y_size, num_channels)
# we only have a single channel
x_train = x_train.reshape(x_train.shape[0], size_x, size_y, 1)
x_test = x_test.reshape(x_test.shape[0], size_x, size_y, 1)
input_shape = (size_x, size_y, 1)

In [0]:
# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (15981, 30, 300, 1)
15981 train samples
3991 test samples


In [0]:
# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
#y_train = y_train.reshape(y_train.shape[0], 1)#keras.utils.to_categorical(y_train, nb_classes)
#y_test = y_test.reshape(y_test.shape[0], 1)#keras.utils.to_categorical(y_test, testnb_classes)


In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

print((y_train.shape))
print((y_test.shape))
# convert class vectors to binary class matrices - this is for use in the
encoder = LabelEncoder()
encoder.fit(GCodes2)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

encoded_Ytest = encoder.transform(y_test)
y_test = np_utils.to_categorical(encoded_Ytest)

print((y_train.shape))
print((y_test.shape))

(15981,)
(3991,)
(15981, 18372)
(3991, 18371)


In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
plt.plot(range(1, 11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

ValueError: ignored